# Importamos librerias - Creamos el dataframe de pericias

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df = df['text'].apply(str)
df1 = df.head(20)
df1

0      \n\nInforma a continuación: \n\nJ.T. 13 \nPER...
1      \n\nJUZG. 40 \n\nPERITO MEDICO PRESENTA INFOR...
2     Expediente Nº CNT 74803/2014 \n \nJUZGADO NACI...
3     JUZGADO NACIONAL DE TRABAJO 59       LAVALLE 1...
4      \n\n \n\nPERITO PRESENTA INFORME \n\n \n\n \n...
5                           JNT N°72 \n\nPERITO MEDI...
6     J. T.  Nº 20              Exp. 9231//2016     ...
7     PERITO MEDICO PRESENTA INFORME \n\nSr. Juez: \...
8     II.- HISTORIA CLINICA \n\nPERITO MEDICO PRESEN...
9     JNT nº 67, Sito en Calle Lavalle nº 1268, Ciud...
10    PERITO MEDICO: PRESENTA PERITACION MEDICA.- \n...
11     \n\nSR. JUEZ: \n\nJUZGADO NACIONAL DEL TRABAJ...
12    PRESENTA DICTAMEN PERICIAL \n\nSeñor Juez: \n\...
13    JUZG. NAC. DEL TRABAJO Nº 12 \n\n             ...
14    SE PRESENTA – HACE SABER: \n\nJuz T 51 \n\n \n...
15    PERITO MÉDICO PRESENTA INFORME PERICIAL\n\nSr....
16    PRESENTA INFORME PERICIAL MÉDICO. \n\nJ.N.T. 1...
17    PERITO MEDICO PRESENTA INFORME PERICIAL \n

## Limpieza de texto

In [3]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
'''
Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
Se eliminan los espacios de sobra
Se eliminan \r, \t, \v, \f, \a
'''
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1.apply(str)
df1 = df1.apply(limpiarTexto1)
#df1

In [4]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('^ ',' ',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[º<>/-]','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

dfLimpio = df1.apply(limpiarTexto2)
dfLimpio

0     Informa a continuacion:\nJ.T. 1PERITO MEDICO P...
1     JUZG. 4PERITO MEDICO PRESENTA INFORME MEDICOPE...
2     Expediente  CNT JUZGADO NACIONAL DE 1 INSTANCI...
3     JUZGADO NACIONAL DE TRABAJO 59 LAVALLE  2P\nPE...
4     PERITO PRESENTA INFORME\nDatos personales\n\nJ...
5     JNT N°7PERITO MEDICA PRESENTA INFORME PERICIAL...
6     J. T.  20 Exp.  R. Saenz Peña . P. 2\nPERITO M...
7     PERITO MEDICO PRESENTA INFORME\nSr. Juez:\nEl ...
8     II. HISTORIA CLINICA\nPERITO MEDICO PRESENTA P...
9     JNT  67, Sito en Calle Lavalle  , Ciudad Auton...
10    PERITO MEDICO: PRESENTA PERITACION MEDICA.\nSe...
11    SR. JUEZ:\nJUZGADO NACIONAL DEL TRABAJO  4PERI...
12    PRESENTA DICTAMEN PERICIAL\nSeñor Juez:\nEl Dr...
13    JUZG. NAC. DEL TRABAJO  1 EXP.  PERICIA MEDICA...
14    SE PRESENTA – HACE SABER:\nJuz T 5Sr. Juez:\nR...
15    PERITO MEDICO PRESENTA INFORME PERICIAL\nSr. J...
16    PRESENTA INFORME PERICIAL MEDICO.\nJ.N.T. 1Señ...
17    PERITO MEDICO PRESENTA INFORME PERICIAL\nJ

In [5]:
# Tercera fase de limpieza
# Eliminamos las lineas que no son de utilidad para el analisis o que van a afectar los resultados del mismo.
# Ejemplo de linea eliminada: las lineas que comienzan con "Se encuentra contestada en....."
import re

def limpiarTexto3(text):
    text = re.sub('[a-z1-9.]+[).-] [s|S]e encuentra contestad[a|o] .+[. \n]','',text)
    text = re.sub('[0-9]+[. ]+[yY]a fue contestado.+[.\n]','',text)
    text = re.sub('[fF]oja [1-9].+\n', '', text)
    text = re.sub('[pP]regunta[ 0-9]+[)].+\n|[rR]espuesta[ 0-9]+[)].+\n','',text)#elimina oraciones comenzadas en preguta/respuesta.
    return text

dfLimpio = dfLimpio.apply(limpiarTexto3)
dfLimpio = dfLimpio.apply(limpiarTexto2)
dfLimpio

0     Informa a continuacion:\nJ.T. 1PERITO MEDICO P...
1     JUZG. 4PERITO MEDICO PRESENTA INFORME MEDICOPE...
2     Expediente  CNT JUZGADO NACIONAL DE 1 INSTANCI...
3     JUZGADO NACIONAL DE TRABAJO 59 LAVALLE  2P\nPE...
4     PERITO PRESENTA INFORME\nDatos personales\nJuz...
5     JNT N°7PERITO MEDICA PRESENTA INFORME PERICIAL...
6     J. T.  20 Exp.  R. Saenz Peña . P. PERITO MEDI...
7     PERITO MEDICO PRESENTA INFORME\nSr. Juez:\nEl ...
8     II. HISTORIA CLINICA\nPERITO MEDICO PRESENTA P...
9     JNT  67, Sito en Calle Lavalle  , Ciudad Auton...
10    PERITO MEDICO: PRESENTA PERITACION MEDICA.\nSe...
11    SR. JUEZ:\nJUZGADO NACIONAL DEL TRABAJO  4PERI...
12    PRESENTA DICTAMEN PERICIAL\nSeñor Juez:\nEl Dr...
13    JUZG. NAC. DEL TRABAJO  1 EXP.  PERICIA MEDICA...
14    SE PRESENTA – HACE SABER:\nJuz T 5Sr. Juez:\nR...
15    PERITO MEDICO PRESENTA INFORME PERICIAL\nSr. J...
16    PRESENTA INFORME PERICIAL MEDICO.\nJ.N.T. 1Señ...
17    PERITO MEDICO PRESENTA INFORME PERICIAL\nJ

In [85]:
#QUISE GUARDAR EL DFLIMPIO SIN LAS PALABRAS CON LONGITUD MENOR A 2.
#texto_nuevo = " "
#for w in dfLimpio:
#    if len(w)>2:
#        texto_nuevo = texto_nuevo + " " + w
#print(texto_nuevo)

# Búsqueda de títulos

In [6]:
# Busqueda de titulos
# Se crea un nuevo dataframe de titulos según distintos criterios

# busca titulos que comienzan con numeros romanos
def buscarTitulosRomanos(text):
    # Expresiones regulares para encontrar los numeros romanos
    tituloNroRomano =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][\w\s]+[:\n]')
    titulosRomanosEncontrados = []

    for m in tituloNroRomano.finditer(text):
        titulosRomanosEncontrados.append(m.group())

    return titulosRomanosEncontrados

# titulosConNroRomano: lista que guarda los titulos que comienzan con numeros romanos   
titulosConNroRomano=[]
for expediente in dfLimpio:
    titulosConNroRomano.append(buscarTitulosRomanos(expediente))

#pprint(titulosConNroRomano[10])

In [ ]:
#NO ESTA BIEN ESTO
import pandas as pd
dfTitulosCompletosRom = pd.DataFrame(titulosConNroRomano)
dfTitulosCompletosRom

In [8]:
#TAMPOCO QUEDA BIEN 
#Se exporta el dataframe a csv
dfTitulosCompletosRom.to_csv('titCompletosRom.csv', index=False, sep=';')

In [9]:
# Busca titulos que comienzan con numeros latinos
def buscarTitulosLatinos(text):
    # Expresion regular para encontrar titulos que comienzan con numeros
    tituloNroLatino =re.compile(r'[1-9][-.)][\w\s]+[:\n]')
    titulosLatinosEncontrados = []

    for m in tituloNroLatino.finditer(text):
        titulosLatinosEncontrados.append(m.group())

    return titulosLatinosEncontrados

# titulosConNroLatino: lista que guarda los titulos que comienzan con numeros latinos
titulosConNroLatino=[]
for expediente in dfLimpio:
    titulosConNroLatino.append(buscarTitulosLatinos(expediente))
pprint(titulosConNroLatino[9])

['1) Datos personales:',
 '2) Motivo de la evaluacion:',
 '3) Antecedentes Heredofamiliares y personales:',
 '5) Ocupacion actual:',
 '7) Examen Fisico\n',
 '8) ESTUDIOS MEDICOS COMPLEMENTARIOS SOLICITADOS:',
 '1) Informe las lesiones padecidas por el actor como consecuencia del\n',
 '3) Indique los estudios que se le realizaron para su diagnostico y\n',
 '4) Indique tiempos de control ambulatorio y de convalecencia que\n',
 '6) Determine la incapacidad transitoria sufrida por el actor y la\n',
 '7) Relate si debera someterse a tratamientos futuros como\n',
 '4. por los fundamentos\n',
 '2) Determine mediante la realizacion de un Psicodiagnostico con\n',
 '4. por los fundamentos expuestos en el Punto\n',
 '9) de la pericial\nmedica\nPERICIAL MEDICA DEMANDADA\n',
 '1.Que tipo de signo sintomatologia presenta el actor en virtud\n',
 '2.Realice un examen exhaustivo en su columna lumbar\n',
 '3. Analisis radiografico de la columna lumbar valorando si existe\n',
 '4. Si considera que el act

In [10]:
# Busca titulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresion regular para encontrar titulos en mayusculas.
    tituloMayusculas =re.compile(r'([1-9]|I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][A-Z -]+[:.\n]')
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los titulos en mayusculas
titulosMayusculas=[]
for expediente in dfLimpio:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))

pprint(titulosMayusculas)


[['V.S.', 'V.S.', '2. TOBILLO:', '4. PSICOENTREVISTA:', 'I.V.', 'V.S.', 'V.S.'],
 ['V.S.',
  'IV) EXAMEN DEL PERITO DE OFICIO.',
  '3) PROSOPOGRAFIA:',
  '4) DATOS PERSONALES:',
  '7) DATOS LABORALES.',
  '2) IMPOTENCIA FUNCIONAL DE SU M.',
  '1) RESUMEN SEMIOLOGICO:',
  '2) DIAGNOSTICO DE LAS SECUELAS\n',
  'V) CONSIDERACIONES MEDICAS:',
  '5) PRONOSTICO Y TRATAMIENTO REQUERIDO EN EL FUTURO:',
  'VII) CONTESTACION DE PUNTOS PERICIALES.',
  'VIII) BIBLIOGRAFIA.',
  'VII) CONCLUSIONES:',
  'V.S.'],
 ['I.T.', 'V.S\n'],
 ['II) A.', 'II) A.', 'V.S.'],
 ['I.T.', 'I.T.', 'V.S.', 'V.S.'],
 [],
 ['V. S.'],
 ['V.S.', 'V.S.'],
 ['II. HISTORIA CLINICA\n',
  '1. ANTECEDENTES PERSONALES\n',
  '2.ANTECEDENTES DE INTERES MEDICO LEGAL\n',
  '3. EXAMEN FISICO:',
  '4. EXAMEN PSIQUICO:',
  'IV. PUNTOS PERICIALES\n',
  'III. ESTUDIOS SOLICITADOS\n',
  '4. III B.',
  'V. CONSIDERACIONES Y CONCLUSIONES MEDICO LEGALES\n'],
 ['V.S.', '8) ESTUDIOS MEDICOS COMPLEMENTARIOS SOLICITADOS:', 'V. S.', 'V.S.'],
 ['V.

#### Eliminacion de stopwords y tokenización.

In [11]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

def limpiarTitulos(text):
    text = text.lower()
    text = re.sub(r'\n','',text)
    text = re.sub(r'[a-z]\.[a-z]\.','',text)
    text = re.sub(r'(i{2,3}|iv|vi{1,3}|ix)','',text)
    text = re.sub(r'\W',' ',text)    
    text = re.sub(r'\d+','',text)
    text = re.sub(r' [a-z] ','',text)
    
    return text

#elimino stopwords y hago una ultima limpieza
titulosMayusculasStop = limpiarTitulos(str(titulosMayusculas))
titulosMayusculasStop = remove_stops(str(titulosMayusculasStop))
titulosConNroLatinoStop = limpiarTitulos(str(titulosConNroLatino))
titulosConNroLatinoStop = remove_stops(str(titulosConNroLatinoStop))
titulosConNroRomanoStop = limpiarTitulos(str(titulosConNroRomano))
titulosConNroRomanoStop = remove_stops(str(titulosConNroRomanoStop))

In [163]:
#GRAFICO DE TORTA

# Títulos en mayúsculas.

#### Unigrama

In [ ]:
import pandas as pd
dfTitulosMayusculas = pd.DataFrame(titulosMayusculasStop,columns=['Tokens'])
dfTitulosMayusculas


In [178]:
#Se exporta el dataframe a un archivo csv
dfTitulosMayusculas.to_csv('titTokenEnMayusculas.csv', index=False, sep=';')

In [ ]:
#Cantidad de veces que se repiten los tokens en los titulos.
import pandas as pd
FrecuenciaTitMay = pd.DataFrame(titulosMayusculasStop)
cantMay = FrecuenciaTitMay.value_counts()
print(cantMay)

In [128]:
#def word_count(titulosMayusculasStop):
 #   import collections
  #  word_freq = collections.defaultdict(int)
   # with open(titulosMayusculasStop) as f:
    #    for l in f:
     #       for w in l.strip().split():  
      #          word_freq[w] += 1
    #return word_freq



In [ ]:

#DATAFRAME DE LOS TITULOS EN MAYUSCULAS CON SUS FRECUENCIAS.
import pandas as pd
dfFrecuenciaTitMay = pd.DataFrame(cantMay,columns=['Frecuencia'])
dfFrecuenciaTitMay

In [ ]:
#No hay valores vacios
dfFrecuenciaTitMay.isna().sum 

#### Bigramas

In [ ]:
#bigrama de títulos con mayúsculas.
bigramaMay=(list(nltk.ngrams(titulosMayusculasStop,2)))
bigramaMay

In [ ]:
import pandas as pd
dfBigramaMay = pd.DataFrame(bigramaMay,columns=['Token1','Token2'])
dfBigramaMay

In [ ]:
#Cantidad de veces que se repiten los bigramas en los titulos con mayúsculas.
import pandas as pd
FrecuenciaBigMay = pd.DataFrame(bigramaMay,columns=['TOKEN1','TOKEN2'])
cantBigMay = FrecuenciaBigMay.value_counts()
print(cantBigMay)

#### Trigramas

In [ ]:
#trigrama de títulos con mayúsculas.
trigramaMay=(list(nltk.ngrams(titulosMayusculasStop,3)))
trigramaMay

In [ ]:
import pandas as pd
dfTrigramaMay = pd.DataFrame(trigramaMay,columns=['TOKEN1','TOKEN2','TOKEN3'])
dfTrigramaMay

In [ ]:
#Cantidad de veces que se repiten los trigramas en los titulos con mayúsculas.
import pandas as pd
FrecuenciaTrigMay = pd.DataFrame(trigramaMay,columns=['TOKEN1','TOKEN2','TOKEN3'])
cantTrigMay = FrecuenciaTrigMay.value_counts()
print(cantTrigMay)

In [162]:
#INTENTO DE HACER UN GRAFICO PERO NO.
#import pandas as pd
#import numpy as np

#dfFrecuenciaTitMay
#ranges = [1,2,3,4,5,6,7,8,9,10]
#print(dfFrecuenciaTitMay['frecuencia'].groupby(pd.cut(dfFrecuenciaTitMay.frecuencia, ranges, right=False)).count())

#### DataFrame de los títulos con números romanos.

#### Unigrama

In [ ]:
import pandas as pd
dfTitulosRomanos = pd.DataFrame(titulosConNroRomanoStop,columns=['Tokens'])
dfTitulosRomanos


In [179]:
#Se exporta el dataframe a un archivo csv
dfTitulosRomanos.to_csv('titTokenRomanos.csv', index=False, sep=';')

In [ ]:
#Cantidad de veces que se repiten los tokens en los titulos.
import pandas as pd
FrecuenciaTitRomanos = pd.DataFrame(titulosConNroRomanoStop)
cantRom = FrecuenciaTitRomanos.value_counts()
print(cantRom)

In [ ]:
#DATAFRAME DE LOS TITULOS CON NUMEROS ROMANOS Y SUS FRECUENCIAS.
import pandas as pd
dfFrecuenciaTitRom = pd.DataFrame(cantRom,columns=['Frecuencia'])
dfFrecuenciaTitRom

#### Bigramas

In [ ]:
#bigrama de los títulos con números romanos.
bigramaRom = (list(nltk.ngrams(titulosConNroRomanoStop,2)))
bigramaRom

In [ ]:
import pandas as pd
dfBigramaRom = pd.DataFrame(bigramaRom,columns=['TOKEN1','TOKEN2'])
dfBigramaRom

In [ ]:
#Cantidad de veces que se repiten los bigramas en los titulos con números romanos.
import pandas as pd
FrecuenciaBigRom = pd.DataFrame(bigramaRom,columns=['TOKEN1','TOKEN2'])
cantBigRom = FrecuenciaBigRom.value_counts()
print(cantBigRom)

#### Trigramas

In [ ]:
#trigrama de los títulos con números romanos.
trigramaRom = (list(nltk.ngrams(titulosConNroRomanoStop,3)))
trigramaRom

In [ ]:
import pandas as pd
dfTrigramaRom = pd.DataFrame(trigramaRom,columns=['TOKEN1','TOKEN2','TOKEN3'])
dfTrigramaRom

In [ ]:
#Cantidad de veces que se repiten los trigramas en los titulos con números romanos.
import pandas as pd
FrecuenciaTrigRom = pd.DataFrame(trigramaRom,columns=['TOKEN1','TOKEN2','TOKEN3'])
cantTrigRom = FrecuenciaTrigRom.value_counts()
print(cantTrigRom)

#### DataFrame de los títulos con números latinos.

#### Unigrama

In [ ]:
import pandas as pd
dfTitulosLatinos = pd.DataFrame(titulosConNroLatinoStop, columns=['Tokens'])

#from IPython.display import HTML, display_html, display
#display(HTML('<h2>Frecuencia de los titulos con números latinos</h2>'))
#display_html(dfTitulosLatinos)
dfTitulosLatinos

In [180]:
#Se exporta el dataframe a un archivo csv
dfTitulosLatinos.to_csv('titTokenLatinos.csv', index=False, sep=';')

In [ ]:
#Cantidad de veces que se repiten los tokens en los titulos.
import pandas as pd
FrecuenciaTitLatinos = pd.DataFrame(titulosConNroLatinoStop)
cantLat = FrecuenciaTitLatinos.value_counts()
print(cantLat)

In [ ]:
#DATAFRAME DE LOS TITULOS CON NUMEROS LATINOS Y SUS FRECUENCIAS.
import pandas as pd
dfFrecuenciaTitLat = pd.DataFrame(cantLat,columns=['Frecuencia'])
dfFrecuenciaTitLat

#### Bigramas

In [ ]:
#bigrama de títulos con números latinos.
bigramaLat=(list(nltk.ngrams(titulosConNroLatinoStop,2)))
bigramaLat

In [ ]:
import pandas as pd
dfBigramaLat = pd.DataFrame(bigramaLat,columns=['TOKEN1','TOKEN2'])
dfBigramaLat

In [ ]:
#Cantidad de veces que se repiten los bigramas en los titulos con números latinos.
import pandas as pd
FrecuenciaBigLat = pd.DataFrame(bigramaLat,columns=['TOKEN1','TOKEN2'])
cantBigLat = FrecuenciaBigLat.value_counts()
print(cantBigLat)

#### Trigramas

In [ ]:
#trigrama de títulos con números latinos.
trigramaLat=(list(nltk.ngrams(titulosConNroLatinoStop,3)))
trigramaLat

In [ ]:
import pandas as pd
dfTrigramaLat = pd.DataFrame(trigramaLat,columns=['TOKEN1','TOKEN2','TOKEN3'])
dfTrigramaLat

In [88]:
#Cantidad de veces que se repiten los trigramas en los titulos con números latinos.
import pandas as pd
FrecuenciaTrigLat = pd.DataFrame(trigramaLat,columns=['TOKEN1','TOKEN2','TOKEN3'])
cantTrigLat = FrecuenciaTrigLat.value_counts()
print(cantTrigLat)

TOKEN1         TOKEN2            TOKEN3           
honorarios     correspondientes  mas                  3
oportunamente  regule            honorarios           3
regule         honorarios        correspondientes     3
hago           reserva           derecho              2
reserva        derecho           accionar             2
                                                     ..
examen         exhausto          columna              1
               completo          funciones            1
evolucion      daños             psiquicosmorales     1
evaluacion     si                actor                1
vengo          nla               presentedenunciar    1
Length: 519, dtype: int64


# Dataframe de todos los titulos

In [ ]:
# crear un df de títulos.
import pandas as pd

dfTitulos = pd.DataFrame()
dfTitulos['latinos'] = titulosConNroLatinoStop
dfTitulos['romanos'] = pd.DataFrame(titulosConNroRomanoStop)
dfTitulos['mayusculas'] = pd.DataFrame(titulosMayusculasStop)
dfTitulos

In [135]:
#INTENTO DE HACER UN GRAFICO PERO NO.
#import pandas as pd
#import numpy as np

#dfFrecuenciaTitMay
#ranges = [1,2,3,4,5,6,7,8,9,10]
#print(dfFrecuenciaTitMay['frecuencia'].groupby(pd.cut(dfFrecuenciaTitMay.frecuencia, ranges, right=False)).count())

In [152]:
#Queria saber cuales son los stopwords que hay 
#from nltk.corpus import stopwords
#stopwords.words('spanish')

In [137]:
#Se importan las librerias necesarias y se toman los datos de muestra
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
dato1 = titulosMayusculasStop
dato2 = titulosConNroRomanoStop
dato3 = titulosConNroLatinoStop

In [138]:
#Convierte los datos de muestra en DataFrame usando Pandas
dfTodosLosTitulos = pd.DataFrame({'Mayúsculas': [dato1], 'Romanos': [dato2], 'Latinos': [dato3]})
print (dfTodosLosTitulos)

                                          Mayúsculas  \
0  [tobillo, psicoentresta, examen, perito, ofici...   

                                             Romanos  \
0  [nla, articulacion, factible, inestabilidad, a...   

                                             Latinos  
0  [tobillo, psicoentresta, nlas, secuelas, tobil...  


In [139]:
#Inicializa el vectorizador
#Se inicializa el vectorizador y se ajusta y transforman los datos
tfidf_vectorizer = TfidfVectorizer()
doc_vec = tfidf_vectorizer.fit_transform(dfTodosLosTitulos)

In [140]:
#Convierte los datos transformados en un marco de datos
dfMarco = pd.DataFrame(doc_vec.toarray().transpose(), index= tfidf_vectorizer.get_feature_names_out())

In [141]:
#Cambia los nombres de las columnas e imprime el resultado
dfMarco.columns = dfTodosLosTitulos.columns
print(dfMarco)

            Mayúsculas  Romanos  Latinos
latinos            0.0      0.0      1.0
mayúsculas         1.0      0.0      0.0
romanos            0.0      1.0      0.0


In [ ]:
#FRECUENCIA DE TODO EL TEXTO (df1 o dfLimpio????)
from sklearn.feature_extraction.text import CountVectorizer 

documento = df1.apply(str)  
# crear la transformación  
vectorizer = CountVectorizer() 
# tokenizar y construir el vocabulario
vectorizer.fit(documento) 
  
print("Vocabulary: ", vectorizer.vocabulary_) 
# codificador de documentos
vector = vectorizer.transform(documento) 

# resumir vector codificado
#print("El documento codificado es:") 
#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

In [ ]:
# r'.\n +'[0-9]+[-. ]+[a-zA-z ]+'

#.\n +'([vV][.-]+ [A-Za-z ]+)|([iI{0,3}][vV]+[.-]+ [A-Za-z ]+)